# Rule Engine DSL

A rule consists of a __Condition__ and an __Action__. A condition demarcates a situation where the concomitant action is to applied.

In [ ]:
#r "nuget:Microsoft.Diagnostics.Tracing.TraceEvent"

Installed Packages Microsoft.Diagnostics.Tracing.TraceEvent, 2.0.74

In [ ]:
open System;
open System.Linq;
open Microsoft.Diagnostics.Tracing;
open Microsoft.Diagnostics.Tracing.Analysis;
open Microsoft.Diagnostics.Tracing.Etlx;
open Microsoft.Diagnostics.Tracing.Session;
open Microsoft.Diagnostics.Tracing.Parsers.Clr;
open Microsoft.Diagnostics.Tracing.Analysis.GC;

## Condition 

In [ ]:
// Format: EventName.Property Condition Value
// For example: GCEnd.AllocationRate LessThan 100

// TODO: Decide if it's better to type?
// Conditioner Event is the name of the type of event.
type ConditionerEvent    = string
// Conditioner Property is the name of the property of the event.
type ConditionerProperty = string

type Conditioner = { ConditionerEvent: ConditionerEvent; ConditionerProperty : ConditionerProperty }

type ConditionType = 
    | LessThan
    | LessThanEqualTo
    | GreaterThan
    | GreaterThanEqualTo
    | Equal
    | NotEqual

type ConditionalValue = double 

type Condition = 
    {  Conditioner      : Conditioner;
       ConditionType    : ConditionType;
       ConditionalValue : ConditionalValue }

#### Parsing Logic

In [ ]:
let parseCondition (conditionAsString : string) : Condition = 

    let splitCondition : string[] = conditionAsString.Split(" ", StringSplitOptions.RemoveEmptyEntries)
    
    // Precondition check
    if splitCondition.Length <> 3
    then failwith("Incorrect format of the condition. Format is: Event.Property Condition ConditionalValue. For example: GCEnd.SuspensionTimeMSec >= 298")
    
    // Condition Event and Property
    let parseConditioner : Conditioner = 
        let splitConditioner : string[] = splitCondition.[0].Split(".", StringSplitOptions.RemoveEmptyEntries)
        let parseConditionEvent : ConditionerEvent = splitConditioner.[0]
        let parseConditionProperty : ConditionerProperty = splitConditioner.[1]

        { ConditionerEvent = parseConditionEvent; ConditionerProperty = parseConditionProperty }

    // Condition Type
    let parseConditionType : ConditionType =
        match splitCondition.[1].ToLower() with
        | ">"  | "greaterthan"                                 -> ConditionType.GreaterThan 
        | "<"  | "lessthan"                                    -> ConditionType.LessThan
        | ">=" | "greaterthanequalto" | "greaterthanorequalto" -> ConditionType.GreaterThanEqualTo
        | "<=" | "lessthanequalto"    | "lessthanorequalto"    -> ConditionType.LessThanEqualTo
        | "="  | "equal"              | "equals"               -> ConditionType.Equal
        | "!=" | "notequal"                                    -> ConditionType.NotEqual
        | _                                                    -> failwith("${splitCondition.[1]} is an unrecognized condition type.")

    // Condition Value
    let parseConditionValue : ConditionalValue =
        let conditionalValueAsString = splitCondition.[2]
        let checkDouble, doubleValue = Double.TryParse conditionalValueAsString 
        if checkDouble then doubleValue
        else failwith($"{doubleValue} isn't a double.")

    { Conditioner = parseConditioner; ConditionType = parseConditionType; ConditionalValue = parseConditionValue }

## Action

In [ ]:
// For example: <Condition>; Print CallStack

type ActionOperator = 
    |  Print

type ActionOperand =
    | Alert
    | CallStack

type Action = { ActionOperator: ActionOperator; ActionOperand: ActionOperand }

#### Parse Action

In [ ]:
let parseAction (actionAsAString : string) : Action = 
    let splitAction : string[] = actionAsAString.Split(" ", StringSplitOptions.RemoveEmptyEntries)

    // ActionOperator
    let parseActionOperator : ActionOperator = 
        match splitAction.[0].ToLower() with
        | "print" -> ActionOperator.Print
        | _       -> failwith($"{splitAction.[0]} is an unrecognized Action Operator.")

    // ActionOperand 
    let parseActionOperand : ActionOperand = 
        match splitAction.[1].ToLower() with
        | "alert"     -> ActionOperand.Alert
        | "callstack" -> ActionOperand.CallStack
        | _           -> failwith($"{splitAction.[1]} is an unrecognized Action Operand.")

    { ActionOperator = parseActionOperator; ActionOperand = parseActionOperand }

## Rule: Combining Conditions and Actions

In [ ]:
type Rule             = { Condition : Condition; Action : Action; OriginalRule : string }
type RuleApplier      = Rule * TraceEvent -> unit
type ConditionChecker = Rule * TraceEvent -> bool

In [ ]:
let applyRule (rule : Rule) (traceEvent : TraceEvent) : unit =

    // Helper fn checks if the condition is met for the traceEvent.
    let checkCondition : bool =
        let condition : Condition = rule.Condition

        // Match the event name.
        let matchEventName (rule : Rule) (traceEvent : TraceEvent): bool = 
            traceEvent.EventName = condition.Conditioner.ConditionerEvent
        
        // Check if the specified payload exists.
        let checkPayload (rule : Rule) (traceEvent : TraceEvent): bool = 
            if traceEvent.PayloadNames.Contains condition.Conditioner.ConditionerProperty then true
            else false

        // Check if the condition matches.
        let checkConditionValue (rule : Rule) (traceEvent : TraceEvent): bool =
            let payload : double   = Double.Parse(traceEvent.PayloadByName(condition.Conditioner.ConditionerProperty).ToString())
            let value   : double   = rule.Condition.ConditionalValue

            match condition.ConditionType with
            | ConditionType.Equal              -> payload = value
            | ConditionType.GreaterThan        -> payload > value
            | ConditionType.GreaterThanEqualTo -> payload >= value
            | ConditionType.LessThan           -> payload < value
            | ConditionType.LessThanEqualTo    -> payload <= value
            | ConditionType.NotEqual           -> payload <> value

        // Match on Event Name, if the payload exists and the condition based on the trace event is met.
        matchEventName rule traceEvent && checkPayload rule traceEvent && checkConditionValue rule traceEvent

    let apply (action : Action): unit = 
        match action.ActionOperator with
        | ActionOperator.Print ->
            match action.ActionOperand with
            | ActionOperand.Alert -> printfn $"Alert!! {rule.OriginalRule} invoked as payload: {traceEvent.PayloadByName(rule.Condition.Conditioner.ConditionerProperty).ToString()}!"
            | ActionOperand.CallStack -> () // TODO: FILL THIS.
    
    if checkCondition = true then apply rule.Action
    else ()

## Parser

The goal is to parse the following types of Rules:

1. ``GCEnd.PauseTimeMSec > 100 : Print CallStack``
2. ``GCEnd.SuspensionDurationMSec IsAnomaly Spike: Print HeapStat``
3. ``GCEnd.PauseDurationMSec >= 100: Print Alert``

### Parser Logic
1. Split on ``:``.
2. First element of the split is the Condition.
   1. Match the Condition Event and Property and associate them with a real type. 
   2. Match the Condition Type and the Value and match them with a real func.
3. Second element of the split is the Action.
   1. Match the Action Operator and the Operator.

### Testing Conditional Parsing

In [ ]:
let testConditional1   = "GCEnd.SuspensionTimeMSec > 100"
let parsedConditional1 = parseCondition testConditional1
display(parsedConditional1)

Conditioner,ConditionType,ConditionalValue
"{ { ConditionerEvent = ""GCEnd"" ConditionerProperty = ""SuspensionTimeMSec"" }: ConditionerEvent: GCEnd, ConditionerProperty: SuspensionTimeMSec }",GreaterThan,100


### Testing Action Parsing

#### Success Case

In [ ]:
let testActionParsing : string = "Print CallStack"
display(parseAction testActionParsing)

ActionOperator,ActionOperand
Print,CallStack


#### Failure Case

In [ ]:
let testActionParsingWithFailure : string = "Print Value"
display(parseAction testActionParsingWithFailure)

Error: System.Exception: Value is an unrecognized Action Operand.
   at FSI_0386.parseAction(String actionAsAString)
   at <StartupCode$FSI_0391>.$FSI_0391.main@()

### Parsing a Rule

In [ ]:
let parseRule (ruleAsString : string) : Rule = 
    let splitRuleAsAString : string[] = ruleAsString.Split(":")
    let condition : Condition = parseCondition splitRuleAsAString.[0]
    let action : Action = parseAction splitRuleAsAString.[1]
    { Condition = condition; Action = action; OriginalRule = ruleAsString }

### Testing Parsing a Rule

In [ ]:
let testRule1 = "GCEnd.SuspensionTimeMSec > 100 : Print CallStack"
let parsedTestRule1 = parseRule testRule1
display(parsedTestRule1)

Condition,Action,OriginalRule
"{ { Conditioner = { ConditionerEvent = ""GCEnd"" ConditionerProperty = ""SuspensionTimeMSec"" } ConditionType = GreaterThan ConditionalValue = 100.0 }: Conditioner: { { ConditionerEvent = ""GCEnd"" ConditionerProperty = ""SuspensionTimeMSec"" }: ConditionerEvent: GCEnd, ConditionerProperty: SuspensionTimeMSec }, ConditionType: GreaterThan, ConditionalValue: 100 }","{ { ActionOperator = Print ActionOperand = CallStack }: ActionOperator: Print, ActionOperand: CallStack }",GCEnd.SuspensionTimeMSec > 100 : Print CallStack


In [ ]:
let testRule2 = "GCEnd.PauseTimeMSec >= 300 : Print Alert"
let parsedTestRule2 = parseRule testRule2
display(parsedTestRule2)

Condition,Action,OriginalRule
"{ { Conditioner = { ConditionerEvent = ""GCEnd"" ConditionerProperty = ""PauseTimeMSec"" } ConditionType = GreaterThanEqualTo ConditionalValue = 300.0 }: Conditioner: { { ConditionerEvent = ""GCEnd"" ConditionerProperty = ""PauseTimeMSec"" }: ConditionerEvent: GCEnd, ConditionerProperty: PauseTimeMSec }, ConditionType: GreaterThanEqualTo, ConditionalValue: 300 }","{ { ActionOperator = Print ActionOperand = Alert }: ActionOperator: Print, ActionOperand: Alert }",GCEnd.PauseTimeMSec >= 300 : Print Alert


## Combining the Trace Log API With The Rule Engine

In [ ]:
let ETL_FILEPATH = @"C:\Users\mukun\OneDrive\Documents\CallstackShmuff.etl\CallstackShmuff.etl" 

let session = new TraceEventSession("TestSession2", ETL_FILEPATH)
let traceLog = TraceLog.OpenOrConvert(ETL_FILEPATH)

let rule : string     = "GC/AllocationTick.AllocationAmount > 107000: Print Alert"
let parsedRule : Rule = parseRule(rule)

let allocationAmountForDevenv =
    traceLog.Events
    |> Seq.filter(fun e -> e.ProcessName = "devenv" && e.EventName.Contains("GC/AllocationTick"))
    |> Seq.take 5
    |> Seq.iter(fun e -> applyRule parsedRule e)

Alert!! GC/AllocationTick.AllocationAmount > 107000: Print Alert invoked as payload: 107376!
Alert!! GC/AllocationTick.AllocationAmount > 107000: Print Alert invoked as payload: 107136!
Alert!! GC/AllocationTick.AllocationAmount > 107000: Print Alert invoked as payload: 107208!
